In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings(action='ignore') # 경고 메시지 무시
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import GradientBoostingClassifier as GBC

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score

In [ ]:
# 레드 와인데이터셋 불러오기
red = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
# 화이트 와인데이터셋 불러오기
white = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

In [ ]:
red.head(2)

In [ ]:
white.head(3)

In [ ]:
red.columns

In [ ]:
red = red.drop(['quality'], axis = 1)
white = white.drop(['quality'], axis = 1)

In [ ]:
red.head(3)

In [ ]:
white.head(3)

In [ ]:
red.isnull().sum()

In [ ]:
white.isnull().sum()

In [ ]:
# 이상치 제거 - boxplot 시각화
def boxplot_vis(data, target_name):
    plt.figure(figsize = (30, 30))
    for col_idx in range(len(data.columns)):
        plt.subplot(6, 2, col_idx+1)
        plt.boxplot(data[data.columns[col_idx]],
                         flierprops = dict(markerfacecolor = 'r',
                                           marker = 'D'))
        plt.title('feature' + '(' + target_name + '):' + data.columns[col_idx],
                  fontsize = 20)
    plt.savefig('/content/drive/MyDrive/Colab Notebooks/figure/boxplot_' + target_name + '.png')
    plt.show()        

In [ ]:
def remove_outlier(input_data):
    q1 = input_data.quantile(0.25)
    q3 = input_data.quantile(0.75)
    iqr = q3 - q1
    minimum = q1 - (iqr*1.5)
    maximum = q3 + (iqr*1.5)
    df_removed_outlier = input_data[(minimum < input_data) & (input_data < maximum)]
    return df_removed_outlier

In [ ]:
boxplot_vis(red, 'red')

In [ ]:
red_prep = remove_outlier(red)

In [ ]:
# 목표변수 할당
red_prep['target'] = 0

In [ ]:
red_prep.isnull().sum()

In [ ]:
red_prep.head(3)

In [ ]:
# 이상치 포함 데이터(이상치 처리 후 NaN) 삭제
red_prep.dropna(axis = 0, how = 'any', inplace = True)
print(f'이상치 포함된 데이터 비율: {round((len(red) - len(red_prep))*100/len(red), 2)}%')

In [ ]:
white_prep = remove_outlier(white)

In [ ]:
white_prep['target'] = 1

In [ ]:
white_prep.isnull().sum()

In [ ]:
white_prep.dropna(axis = 0, how = 'any', inplace = True)
print( f'이상치 포함된 비율: {round((len(white) - len(white_prep))*100/len(white), 2)}%')

In [ ]:
# 데이터 저장
red_prep.to_csv('/content/drive/MyDrive/Colab Notebooks/red_prep.csv')
white_prep.to_csv('/content/drive/MyDrive/Colab Notebooks/white_prep.csv')

In [ ]:
# 데이터 병합
# Now combine RED WINE and WHITE WINE data using concat module on Pandas
df = pd.concat([red_prep, white_prep], axis = 0)
df.head()

In [ ]:
# Save combined dataset
df.to_csv('/content/drive/MyDrive/Colab Notebooks/wine_combined.csv')

In [ ]:
# Check the wine ratio
print(df.target.value_counts(normalize=True))
print(df.target.value_counts())

In [ ]:
# 설명변수별 목표변수 간의 관계 시각화
# 설명변수 1개씩 반복문을 통해 선정한 후 레드 와인과 화이트 와인 각각에 해당하는 데이터를 histogram으로 표현

In [ ]:
x = df[df.columns.difference(['target'])]
# 설명변수명 리스트
feature_name = x.columns
plt.figure(figsize = (10, 25))
for col_idx in range(len(feature_name)):
      # 6행 2열 서브플롯에 각 feature boxplot 시각화
      plt.subplot(6, 2, col_idx+1)
      plt.hist(df[df['target'] == 0][feature_name[col_idx]],
               label = 'Red wine',
               alpha = 0.5)
      plt.hist(df[df['target'] == 1][feature_name[col_idx]],
               label = 'White wine',
               alpha = 0.5)
      plt.legend()
      plt.title('Feature: ' + feature_name[col_idx],
                fontsize = 10)
plt.show()

In [ ]:
# Data scaling
scaler = StandardScaler()
# 목표변수 분리 & 설명변수 데이터 스케일링
y = df['target']
x_scaled = scaler.fit_transform(x)

In [ ]:
# Split data set
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y,
                                                    test_size = 0.3,
                                                    random_state = 123)

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
y_test.value_counts(normalize=True)

In [ ]:
# Classefier 모델링 함수
# 기본 모델 학습 함수
def modeling_uncustomized(algorithm, x_train, y_train, x_test, y_test):
    # 하이퍼파라미터 조정 없이 모델 학습
    uncustomized = algorithm(random_state = 1234)
    uncustomized.fit(x_train, y_train)
    # train and test data 설명력
    train_score_before = uncustomized.score(x_train, y_train).round(3)
    print(f'학습 데이터셋 정확도: {train_score_before}')
    test_score_before = uncustomized.score(x_test, y_test).round(3)
    print(f'테스트 데이터셋 정확도: {test_score_before}')
    return train_score_before, test_score_before


In [ ]:
# 하이퍼파라미터별 모델 성능 시각화 함수
def optimi_visualization(algorithm_name, x_values, train_score, test_score,
                         xlabel, filename):
      # 하이퍼 파라미터 조정에 따른 학습 데이터셋 기반 모델 성능 추이 시각화
      plt.plot(x_values, train_score, linestyle = '-', label = 'train score')
      # 하이퍼 파라미터 조정에 따른 테스트 데이터셋 기반 모델 성능 추이 시각화
      plt.plot(x_values, test_score, linestyle = '--', label = 'test score')
      plt.ylabel('Accuracy(%)')   # y축 레이블
      plt.xlabel(xlabel)          # x축 레이블
      plt.legend()                # 범례표시

In [ ]:
# 모델 최적화 함수 : 학습할 트리 모델 개수 선정
def optimi_estimator(algorithm, algorithm_name, x_train, y_train, x_test, y_test,
                     n_estimator_min, n_estimator_max):
    train_score = [] ; test_score = []
    para_n_tree = [n_tree*5 for n_tree in range(n_estimator_min, n_estimator_max)]

    for v_n_estimators in para_n_tree:
        model = algorithm(n_estimators = v_n_estimators, random_state = 1234)
        model.fit(x_train, y_train)
        train_score.append(model.score(x_train, y_train))
        test_score.append(model.score(x_test, y_test))
    # 트리 개수에 따른 모델 성능 저장
    df_score_n = pd.DataFrame({'n_estimators': para_n_tree, 'TrainScore': train_score, 'TestScore': test_score})
    # 트리 개수에 따른 모델 성능 추이 시각화 함수 호출
    optimi_visualization(algorithm_name, para_n_tree, train_score, test_score, 'The number of estimator', 'n_estimator')
    print(round(df_score_n, 4))

#### 모델 최적화 함수: 최대 깊이 선정
   * 다음은 모델이 학습할 트리별 최대 깊이를 결정하기 위한 함수입니다.
   * 전달받은 최대 깊이의 최소값부터 깊이를 1씩 최대 깊이의 최대값까지 늘려가며 모델의 성능을 평가합니다.
   * 모델 성능은 앞서 작성한 시각화 함수에 전달하여 하이퍼파라미터에 따른 성능 변화 추이를 시각황합니다.

In [ ]:
def optimi_maxdepth(algorithm, algorithm_name, x_train, y_train, x_test, y_test,
                    depth_min, depth_max, n_etsimator):
    train_score = []; test_score = []
    para_depth = [depth for depth in range(depth_min, depth_max)]
    
    for v_max_depth in para_depth:
        # 의사결정나무 모델의 경우 트리 갯수를 따로 설정하지 않기 때문에, RFC, GBC와 분리하여 모델링
        if algorithm == DTC:
            model = algorithm(max_depth = v_max_depth,
                              random_state = 1234)
        else:
            model = algorithm(max_depth = v_max_depth,
                              n_estimators = n_estimator,
                              random_state = 1234)

        model.fit(x_train, y_train)
        train_score.append(model.score(x_train, y_train))
        test_score.append(model.score(x_test, y_test))

    # 최대 깊이에 따른 모델 성능 저장
    df_score_n = pd.DataFrame({'depth': para_depth,
                               'TrainScore': train_score,
                               'TestScore': test_score})
    # 최대 깊이에 따른 모델 성능 추이 시각화 함수 호출
    optimi_visualization(algorithm_name, para_depth, train_score, test_score, "The number of depth", "n_depth")
    print(round(df_score_n, 4))

#### 모델 최적화 함수: 분리 노드의 최소 자료 수 선정
* 노드를 분리하기 위한 최소 자료 수를 결정하기 위한 함수
* 전달받은 분리 노드의 최소 자료 수의 최소값부터 깊이를 2씩 
* 분리 노드의 최소 자료 수의 최대값까지 늘려가며 모델의 성능을 평가합니다
* 모델 성능은 앞서 작성한 시각화 함수에 전달하여 하이퍼파라미터에 따른 성능 변화 추이를 시각화합니다. 

In [ ]:
def optimi_minsplit (algorithm, algorithm_name, x_train, y_train, x_test, y_test,
                     n_split_min, n_split_max, n_estimator, n_depth ):
    train_score = []; test_score = []
    para_split = [n_split*2 for n_split in range(n_split_min, n_split_max)]
    for v_min_samples_split in para_split:
        # 의사결정나무 모델의 경우 트리 개수를 따로 설정하지 않기 때문에 
        # RFC, GBC와 분리하여 모델링
        if algorithm == DTC:
            model = algorithm(min_samples_split = v_min_samples_split,
                              max_depth = n_depth,
                              random_state = 1234)
        else:
            model = algorithm(min_samples_split = v_min_samples_split,
                              n_estimators = n_estimator,
                              max_depth = n_depth,
                              random_state = 1234
                              )
        model.fit(x_train, y_train)
        train_score.append(model.score(x_train, y_train))
        test_score.append(model.score(x_test, y_test))

    # 분리 노드의 최소 자료 수에 따른 모델 성능 저장
    df_score_n = pd.DataFrame({'min_samples_split': para_split,
                               'TrainScore': train_score,
                               'TestScore': test_score
                               })
    # 분리 노드의 최소 자료 수에 따른 모델 성능 추이 시각화 함수 호출
    optimi_visualization(algorithm_name, para_split, train_score, test_score,
                         'The minimum number of samples required to split an internal node',
                         'min_samples_split'
                         )
    print(round(df_score_n, 4))

#### 모델 최적화 함수: 잎사귀 노드의 최소 자료 수 선정
   * 다음은 잎사귀 노드 내 최소 자료 수를 결정하기 위한 함수입니다.
   * 전달받은 잎사귀 노드의 최소 자료 수의 최소값부터 깊이를 2씩
   * 잎사귀 노드의 최소 자료 수의 최대값까지 늘려가며 모델의 성능을 평가합니다.
   * 모델 성능은 앞서 작성한 시각화 함수에 전달하여 하이퍼파라미터에 따른 성능 변화 추이를 시각화합니다. 

In [ ]:
def optimi_minleaf(algorithm, algorithm_name, x_train, y_train, x_test, y_test,
                   n_leaf_min, n_leaf_max, n_estimator, n_depth, n_split):
    train_score = []; test_score = []
    para_leaf = [n_leaf*2 for n_leaf in range(n_leaf_min, n_leaf_max)]

    for v_min_samples_leaf in para_leaf:
        # 의사결정나무 모델의 경우 트리 개수를 따로 설정하지 않기 때문에 RFC, GBC와 분리하여 모델링 
        if algorithm == DTC:
            model = algorithm(min_samples_leaf = v_min_samples_leaf,
                              max_depth = n_depth,
                              min_samples_split = n_split,
                              random_state = 1234 )
        else:
            model = algorithm(min_samples_leaf = v_min_samples_leaf,
                              n_estimators = n_estimator,
                              max_depth = n_depth,
                              min_samples_split = n_split,
                              random_state = 1234)
        model.fit(x_train, y_train)
        train_score.append(model.score(x_train, y_train))
        test_score.append(model.score(x_test, y_test))

    # 잎사귀 노드의 최소 자료 수에 따른 모델 성능 저장
    df_score_n = pd.DataFrame({'min_sampels_leaf': para_leaf,
                               'TrainScore': train_score,
                               'TestScore': test_score
                               })
    # 잎사귀 노드의 최소 자료 수에 따른 모델 성능 저장
    df_score_n = pd.DataFrame({'min_samples_leaf': para_leaf, 'TrainScore': train_score, 'TestScore': test_score})
    # 잎사귀 노드의 최소 자료 수에 따른 모델 성능 추이 시각화 함수 호출
    optimi_visualization(algorithm_name, para_leaf, train_score, test_score,
                         "The minimum number of samples required to be at a leaf node", 
                         "min_samples_leaf")
    print(round(df_score_n, 4))

#### 최종 모델 학습
   * 앞서 구한 최적의 하이퍼파라미터 기반으로 최종 모델을 학습합니다.
   * 학습한 모델을 pickle 모듈을 통해 저장합니다.
   * 모델 성능 평가를 위해 평가지표로서 Accuracy, Precision, Recall, F1 score, Confusion Matrix를 활용합니다.
   * 마지막으로 변수별 중요도를 산출하고 시각화합니다.

In [ ]:
def model_final(algorithm, algorithm_name, feature_name, x_train, y_train, x_test, y_test,
                n_estimator, n_depth, n_split, n_leaf):
    # 의사결정나무 모델의 경우 트리 개수를 따로 설정하지 않기 때문에 RFC, GBC와 분리하여 모델링
    if algorithm == DTC:
        model = algorithm(random_state = 1234,
                          min_samples_leaf = n_leaf,
                          min_samples_split = n_split,
                          max_depth = n_depth)
    else:
        model = algorithm(random_state = 1234,
                          n_estimators = n_estimator,
                          min_samples_leaf = n_leaf,
                          min_samples_split = n_split,
                          max_depth = n_depth)
    # 모델 학습
    model.fit(x_train, y_train)
    # 모델 저장
    model_path = '/content/drive/MyDrive/Colab Notebooks/model/'
    model_filename = 'wine_classification_' + algorithm_name + '.pkl'
    with open(model_path + model_filename, 'wb') as f:
        pickle.dump(model, f)
    print(f'최종 모델 저장 완료! 파일 경로: {model_path + model_filename}\n')

    # 최종 모델의 성능 평가
    train_acc = model.score(x_train, y_train)
    test_acc = model.score(x_test, y_test)
    y_pred = model.predict(x_test)
    print(f'Accuracy: {accuracy_score(y_test, y_pred):.3f}')    # 정확도
    print(f'Precision: {precision_score(y_test, y_pred):.3f}')  # 정밀도
    print(f'Recall: {recall_score(y_test, y_pred):.3f}')        # 재현율
    print(f'F1-score: {f1_score(y_test, y_pred):.3f}')          # F1 스코어

    # Confusion matrix
    plt.figure(firsize = (30, 30))
    plot_confusion_matrix(model,
                          x_test, y_test,
                          include_values = True,
                          display_labels = ['Red', 'White'],    # 목표변수 이름
                          cmap = 'Pastel1')                     # 컬러맵
    plt.savefig('/content/drive/MyDrive/Colab Notebooks/figure/' + algorithm_name + '_confusion_matrix.png')  # 혼동행렬
    plt.show()

    # 변수 중요도 산출
    dt_importance = pd.DataFrame()
    dt_importance['Feature'] = feature_name       # 설명변수 이름
    dt_importance['Importance'] = model.feature_importances_    # 설명변수 중요도 산출

    # 변수 중요도 내림차순 정렬
    dt_importance.sort_values('Importance', ascending = False, inplace = True)
    print(dt_importance.round(3))
    # 변수 중요도 시각화
    coordinates = range(len(dt_importance))               # 설명변수 개수만큼 bar 시각화
    plt.barh(y = coordinates, width = dt_importance['Importance'])
    plt.yticks(coordinates, dt_importance['Feature'])     # y축 눈금별 설명변수 이름 기입
    plt.xlabel('Feature Importance')                      # x축 이름
    plt.ylabel('Features')                                # y축 이름
    plt.savefig('/content/drive/MyDrive/Colab Notebooks/figure/' + algorithm_name + '_feature_importance.png')   # 변수 중요도 그래프 저장

#### Random Forest Classifier 기반 분류 모델 성능 평가

   * 라이브러리를 import 할 때 RandomForestClassifier를 RFC로 치환하였습니다.
   * 본 포스팅에서는 위와 같은 플로우에서 학습할 알고리즘 종류만 바꿔 추후에 활용하기 위해 algorithm이라는 객체에 알고리즘을 할당하는 방식을 사용하였습니다.
   * 즉, Gradient boosting을 사용한다면, algorithm 객체에 GBC를 할당하면 되는 것이죠.

In [ ]:
# Random Forest Clasifier 
algorithm = RFC
algorithm_name = 'rfc'

#### 기본 모델 학습
   * 하이퍼파라미터 튜닝 없이 기본 모델을 학습시켜 봅니다

In [ ]:
train_acc_before, test_acc_before = modeling_uncustomized(algorithm, x_train, y_train, x_test, y_test)

#### 모델 최적화: (1)학습할 트리 개수 선정
   * Random Foest 모델이 학습할 최적의 트리 개수를 선정해 보겠습니다.

In [ ]:
n_estimator_min = 1
n_estimator_max = 31
optimi_estimator(algorithm, algorithm_name, x_train, y_train, x_test, y_test,
                 n_estimator_min, n_estimator_max)

   * 트리 개수는 많을수록 과적합 방지에 유리합니다. 따라서 트리 개수가 많고 학습 데이터 기반 모델 정확도와 테스트 데이터 기반 모델 정확도의 차이가 적은 값으로 선정하는 것으로 좋습니다. 그래프보다 더욱 정밀하게 하이퍼파라미터별 모델 성능을 알아보기 위해 데이터프레임 형태로도 추이를 출력해 봤습니다
   * 트리 개수가 30개 일 때 테스트 데이터 기반 모델 정확도가 가장 높고, 트리 갯수가 더 많아져도 성능에 차이가 없다는 점에, 최적의 트리 개수는 30으로 설정하겠습니다.

In [ ]:
n_estimator = 30

#### 모델 최적화: (2) 최대 깊이
   * Random Forest 모델의 최대 깊이를 선정해 보겠습니다.

In [ ]:
depth_min = 1
depth_max = 21
optimi_maxdepth(algorithm, algorithm_name, x_train, y_train, x_test, y_test,
                depth_min, depth_max, n_estimator                
                )

   * 최대 깊이는 적을 수록 과적합 방지에 유리합니다. 따라서 최대 깊이는 적고 학습 데이터 기반 모델 정확도와 테스트 데이터 기반 모델 정확도의 차이가 적은 값으로 선정하는 것이 좋습니다. 아래 그림11과 같이 최대 깊이 값에 따른 모델 성능 추이를 데이터 프레임 형태로 출력해 봤습니다.
   * 테스트 데이터 기반 모델 정확도가 점차 증가하다가 감소하기 시작하는 구간으로, 최대 깊이 6을 최적의 값으로 선정하였습니다.

In [ ]:
n_depth = 6

#### 모델 최적화: (3) 분리 노드의 최소 자료 수

   * 다음으로 노드를 분리할 때 최소 자료 수의 최적의 값을 선정해 보겠습니다.

In [ ]:
n_split_min = 1
n_split_max = 101
# 데이터프레임 행 최대 100개까지 반드시 출력
pd.set_option('display.max_row', 100)
optimi_minsplit(algorithm, algorithm_name,
                x_train, y_train, x_test, y_test,
                n_split_min, n_split_max, n_estimator, n_depth)

   * 분리 노드의 최소 자료 수에 따른 학습 데이터 및 테스트 데이터에서의 모델 성능은 아래 참고

In [ ]:
# 분리 노드의 최소 자료수는 많을수록 과적합 방지에 유리, 분리 노드의 최소 자료수는 많게 하되 학습 데이터 기반 모델 정확도와 테스트 데이터 기반 모델 정확도의 차이가 적은 값으로 선정하는 것이 좋다
n_split = 66

#### 모델 최적화: (4) 잎사귀 노드의 최소 자료 수

In [ ]:
n_leaf_min = 1
n_leaf_max = 51
optimi_minleaf(algorithm, algorithm_name, 
               x_train, y_train, x_test, y_test, 
               n_leaf_min, n_leaf_max, n_estimator, n_depth, n_split)

#### 해석
   * 학습 데이터셋 기반 정확도와 테스트 데이터셋 기반 정확도 간의 차이가 적은 잎사귀 노드의 최소 자료 수 선정
   * 잎사귀 노드의 최소 자료 수가 클수록 분류 기준이 엄격해져 과대적합 방지가 용이함
   * 테스트 데이터셋에서 모델 성능이 증가하다가 감소하기 직전인 최적의 이팟귀 노드의 최소 자료 수 선정

In [ ]:
n_leaf = 20

#### 최종 모델 학습

In [ ]:
model_final(algorithm, algorithm_name, feature_name,
            x_train, y_train, x_test, y_test,
            n_estimator, n_depth, n_split, n_leaf)

In [ ]:
!pip install selenium

#### 해석
   * 최종 학습한 모델의 정확도가 99.8%, f1 score 역시 0.999인 만큼 분류 성능이 우수한 것을 알 수 있음
   * 변수별 중요도 산출 결과, chlorides(염화물), total sulfur dioxide(총 이산화황), volatile acidity(휘발성산), density(밀도)가 분류에 있어 가장 중요한 4가지 변수임